In [ ]:
# Supress Warnings
import warnings
warnings.filterwarnings('ignore')

# Data Science
import pandas as pd
import numpy as np

# Machine Learning
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

# Others
from sklearnex import patch_sklearn
patch_sklearn()

In [ ]:
df = pd.read_csv('2011-2022_mergedcrimerate.csv')
df = df.drop(['Month', 'LSOA code', 'Count', 'AS Score', 'Inflation_rate', 'Indoors Score'], axis=1)
df = df.sample(n = 1000000, random_state = 2)

In [ ]:
X = df.drop(columns=['Crime Rate']).values
y = df['Crime Rate'].values
# Choose any random state
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=17)

In [ ]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
#with parallel_backend('threading', n_jobs=15):

    # Initialize a Random Forest Regressor Model
rf = RandomForestRegressor(n_jobs=-1, verbose=3)

                # Initialize a parameter list that will be iterated in the grid search
forest_params = [{'max_features': ['sqrt', 'log2', 'auto'], 
                    'n_estimators': [50, 100, 150],
                    'max_depth':[16, 20, 24, 28, 32],
                    'bootstrap':[True, False]
                    }]

                # Initiate a grid search model that takes in,
                # - Rnadom Forest classifier model
                # - List of parameters
                # - Cross validation value. It is set to 10 in this case to ensure a robust model
                # - n_jobs is set to 15 to ensure the model runs parallely across all threads in the processor
                # - verbose is set to 3 to get a fairly good description on each loop
                # - scoring metric is set to R2
reg = GridSearchCV(rf, forest_params, cv = 5, scoring='r2', verbose=3)

                # Use the initialized grid search model to train the split data
reg.fit(X_train, y_train)

In [ ]:
rfgrid_results = pd.DataFrame(reg.cv_results_)
rfgrid_results.to_csv('rfgrid_results_sklearnex_1mil.csv', index=False)